In [1]:
%time
# -*- encoding: utf-8 -*-
'''
@File    :   tfidf.py
@Time    :   2020/10/05 22:09:15
@Author  :   DataMagician 
@Version :   1.0
@Contact :   408903228@qq.com
'''

# here put the import lib

import numpy as np
#from numpy import argwhere, zeros, ndarray, array, log, ones
#from data_preprocessing_module import word_punct_tokenizer_for_chinese_function
from tqdm import tqdm
from jieba import cut as jiebacut
from jieba import cut_for_search as jieba_cut_for_search
#from numpy import ndarray, array
from cupy import  zeros, ndarray, array, log, ones , ndarray , array , asnumpy




need_type = list, tuple, ndarray, set

'''加载标准停用词（标点符号）'''
base_stopwords = ['.', '!', '?', '＂', '＃'
    , '＄', '％', '＆', '＇', '（', '）', '＊'
    , '＋', '，', '－', '／', '：', '；', '＜'
    , '＝', '＞', '＠', '［', '＼', '］', '＾'
    , '＿', '｀', '｛', '｜', '｝', '～', '｟'
    , '｠', '｢', '｣', '､', '\u3000', '、'
    , '〃', '〈', '〉', '《', '》', '「', '」'
    , '『', '』', '【', '】', '〔', '〕', '〖'
    , '〗', '〘', '〙', '〚', '〛', '〜', '〝'
    , '〞', '〟', '〰', '〾', '〿', '–', '—'
    , '‘', '’', '‛', '“', '”', '„', '‟', '…'
    , '‧', '﹏', '﹑', '﹔', '·', '.', '!'
    , '?', '"', '#', '$', '%', '&', "'", '('
    , ')', '*', '+', ',', '-', '/', ':', ';'
    , '<', '=', '>', '@', '[', '\\', ']', '^'
    , '_', '`', '{', '|', '}', '~']


# TODO 中英文字符判断
def is_chinese_function(uchar) -> chr:
    '''
     判断一个unicode是否是汉字
    Args:
        uchar: chart形式的字符
    Returns:

    '''
    if uchar >= u'\u4e00' and uchar <= u'\u9fa5':
        return True
    else:
        return False


def is_number_function(uchar) -> chr:
    '''
    判断一个unicode是否是数字
    Args:
        uchar:  chart形式的字符

    Returns:

    '''
    if uchar >= u'\u0030' and uchar <= u'\u0039':
        return True
    else:
        return False


def is_alphabet_function(uchar) -> chr:
    '''
    判断一个unicode是否是英文字母
    Args:
        uchar: chart形式的字符
    Returns:

    '''
    """

    """
    if (uchar >= u'\u0041' and uchar <= u'\u005a') or (uchar >= u'\u0061' and uchar <= u'\u007a'):
        return True
    else:
        return False


def is_othe_function(uchar) -> chr:
    '''
    判断是否非汉字，数字和英文字符
    Args:
        uchar: chart形式的字符

    Returns:

    '''
    if not (is_chinese_function(uchar) or is_number_function(uchar) or is_alphabet_function(uchar)):
        return True
    else:
        return False


def character_type_token(original) -> str:
    '''

    Args: 字符串形式的文章
        original:

    Returns:

    '''
    '''
    不同字符类型分割
    '''
    make = [0]
    diff = []
    n = 0
    temp = ""
    for char in original:
        if is_chinese_function(char):
            n = 0
        elif is_number_function(char):
            n = 1
        elif is_alphabet_function(char):
            n = 2
        elif is_othe_function(char):
            n = 3
        else:
            n = 4
        make.append(n)
        if (make[-1] - make[-2]) == 0:
            diff.append(char)
        else:
            diff.append("|")
            diff.append(char)
    return "".join(diff).split("|")


# TODO 文章列表预处理函数
def context_function(paper_list) -> (list, set, tuple):
    '''
    连接上下文本列表
    Args: 文章列表
        paper_list:

    Returns:

    '''
    return "".join(paper_list)


def tokenize_chinese_function(original) -> str:
    '''
    中文分词
    Args:
        original: 一段文章字符串
    Returns: 分词的列表
    '''
    tokens = []
    for iter in jieba_cut_for_search(context_function(character_type_token(original))):
        temp = ''
        number = 0
        iters = (iter.lower() if is_alphabet_function(iter) else iter)
        n = 0
        for char in iters:
            char = str(ord(char))
            n += 1 
            temp+=char
        tokens.append(int(temp))
    return tokens


def word_punct_tokenizer_for_chinese_function(article_list: list
                                              , filter_stop_words=False) -> (list, tuple, ndarray, tuple, dict):
    '''

    Args: 对文章列表分词(中文优先)
        article_list: 文章列表
        filter_stop_words: 是否清理词不必要的停用词
        True是过滤基础停用词，Flase是不过滤停用词，
        如果是 list,tuple,dict,set,ndarray等可以
        "in" 判断的结构则过滤定义的停用词

    Returns:
    '''
    m = len(article_list)
    if filter_stop_words == True:
        return {paper_num: filter_stop_words_fumction(tokenize_chinese_function(paper)) for paper, paper_num in
                zip(article_list, range(m))}
    elif filter_stop_words == False:
        return {paper_num: tokenize_chinese_function(paper) for paper, paper_num in zip(article_list, range(m))}
    elif isinstance(filter_stop_words, (list, tuple, dict, ndarray, set)):
        return {
            paper_num: filter_stop_words_fumction(tokenize_chinese_function(paper), stop_words_dict=filter_stop_words)
            for paper, paper_num in
            zip(article_list, range(m))}


def filter_stop_words_fumction(words_list: (list, ndarray)
                               , stop_words_dict=base_stopwords) -> (list, tuple, set):
    '''
    过滤停用词
    Args:
        words_list: 需要过滤的词列表
        stop_words_dict: 停用词表

    Returns: 过滤停用词后的词列表

    '''
    return [word for word in words_list if word not in stop_words_dict]



# TODO : 分步TF算法
def tf_function(original_list: (need_type), word_vector) -> need_type:
    '''

    Args:
        paper_words: 文章列表
        word_vector: 词汇表

    Returns:

    '''
    m = len(original_list)
    init_TF = zeros(m)
    for word in original_list:
        if word in word_vector:
            index_ = argwhere(word_vector == word)[0][0]
            init_TF[index_] += 1
    return init_TF


# TODO ：分步IDF算法
def idf_function(paper_words_list, word_vector):
    '''

    Args:
        paper_words_list: 文章列表
        word_vector: 词汇表

    Returns:

    '''
    m = word_vector.size
    init_IDF = zeros(m)
    N = paper_words_list.shape
    n = -1
    for word in word_vector:
        n += 1
        for paper_arr in paper_words_list:
            if word in paper_arr:
                init_IDF[n] += 1
    return np.log(N / (init_IDF + 1))


# TODO : 一次训练的整个训练TFIDF词向量
def data_preprocessing_for_tfidf_function(original: (list, tuple, ndarray, set)
                                          , filter_stop_words=True) -> (bool, list, tuple, ndarray, set):
    '''

    Args:
        paper_words_list:
        word_vector:

    Returns:

    '''
    word_punct_tokenizer = word_punct_tokenizer_for_chinese_function(original
                                                                     , filter_stop_words=filter_stop_words)
    vocabulary = []
    for No, paper_tokens in word_punct_tokenizer.items():
        vocabulary += paper_tokens
    empty_words_dictionay = dict(zip(vocabulary, zeros(len(vocabulary))))
    return {"word_punct_tokenizer": word_punct_tokenizer
        , "vocabulary_set": set(vocabulary)
        , "empty_words_dictionay": empty_words_dictionay}


def TFIDF_function(original_list: (list, ndarray, set, tuple)
                   , filter_stop_words=True):
    '''
    Args:
        word_punct_tokens: 分词后的文章列表
        vocabulary: 词汇表

    Returns: tf 矩阵 横向量为词汇表

    

    '''
    word_punct_tokenizer = data_preprocessing_for_tfidf_function(original_list, filter_stop_words=filter_stop_words)
    word_punct_tokens, vocabulary = word_punct_tokenizer["word_punct_tokenizer"] \
        , word_punct_tokenizer["vocabulary_set"]
    m, n = len(word_punct_tokens), len(vocabulary)
    init_TF = zeros((m, n))
    init_IDF = zeros(n)
    init_counter_words_for_each_document = zeros(m)
    
    for No, paper_tokens in tqdm(word_punct_tokens.items(),"tfidf训练"):
        vocabulary_of_each_document = len(paper_tokens)
        init_TF += array([paper_tokens.count(word) for word in vocabulary])
        init_IDF += array([word in paper_tokens and 1 or 0 for word in vocabulary])
        init_counter_words_for_each_document[No] = vocabulary_of_each_document

    TF = (init_TF.T / init_counter_words_for_each_document).T

    IDF = log(m / (init_IDF + 1))

    return {"TF-IDF": TF * IDF, "TF": TF, "IDF": IDF, "init_TF": init_TF[0], "words_counter": init_IDF.dot(ones(n)),
            "document_count": m,
            "vocabulary_from_TF-IDF": vocabulary}  # ,"counter_vector":init_IDF


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs


ModuleNotFoundError: No module named 'cupy'

In [ ]:
case_text = [["""   职位职责：
负责字节跳动多条产品线（抖音、今日头条、直播、游戏等）的风控安全研究及运营工作，持续和黑灰产斗智斗勇，打击作弊行为。
1、情报监控体系建设：深入调查黑产市场，建立有效监控手段，持续进行作弊情报搜集与分析，为业务提供风险舆情预警；
2、风险治理：分析案例进行策略优化，驱动产品安全机制完善与升级，并能量化风险拦截的价值；
3、黑灰产行业研究：跟踪研究黑产发展变化趋势，输出分析调研报告。"""],

["""职位要求：
1、对业务风险、数据有敏锐度，能够准确描述和识别作弊特征，擅长从大量信息中发现有价值的关键点；
2、对业界主要的安全风控知识有一定了解，能够有效的收集开源情报信息，能够独立运用各种手段进行安全分析，并应用到实际产品中；
3、拥有强烈的责任心和团队合作精神，出色的学习能力；
4、具备强烈的好奇心和自我驱动力，喜欢接受挑战，追求**；
5、具备风控、反作弊、反欺诈、安全及相关领域工作经验或熟悉黑产链条的运作模式或各类作弊手段的优先，有技术背景的优先。"""],
["""        职位职责：
1、情报监控体系建设，深入调查黑产市场，建立有效监控手段，持续进行作弊情报搜集与分析，并输出对抗方案；
2、黑产行业研究：跟踪研究黑产发展变化趋势，定期输出分析调研报告；
职位要求：
1、3年以上风控、反作弊、反欺诈、安全及相关领域工作经验，熟悉黑产链条的运作模式和各类作弊手段，有技术背景优先；
2、对业界主要的安全技术有一定了解，能够有效的收集开源情报信息，能够独立运用各种手段进行安全分析，并应用到实际产品中；
3、拥有强烈的责任心和团队合作精神，出色的学习能力；
4、具备强烈的好奇心和自我驱动力，喜欢接受挑战，追求**。"""],["""        岗位职责：
1、负责分析用户反馈的内容，并根据反馈内容，给出相应的处理方案；
2、负责用反缺陷原因分析，有效挖掘质量提升的点并推进解决.；
3、负责对外包人员进行日常管理及工作指引"""],[""" 任职要求：
1、1年以上地图行业工作经验，本科及以上学历，地图数据分析等经验者优先；
、主动沟通，勤于思考，能够快速掌握处理用反问题的标准和方法，并根据实际工作，对标准和方法优化迭代，能快速掌握新的知识和技能；
3、具备良好的数据分析能力，善于总结问题并能积极的进行过程优化；
4、熟练使用office办公软件；"""],["""工作职责:
1、负责威胁情报生产的相关技术研发；
2、进行网络威胁情报的收集，跟踪国内外的安全动态、威胁情报、安全漏洞等；
3、阅读和翻译国外情报资料，负责情报分析研判，挖掘情报价值，提供及时的可行动的建议措施。
岗位要求:
1、熟悉威胁情报的获取、分析和挖掘，跟踪了解最新的攻击手法；
2、能够独立进行各类安全日志分析与安全运营，并对各种安全事件的优先级和相应机制有比较深的理解，可以进行漏洞和**木马分析；
3、有良好的文字能力和英文基础，熟练阅读英文网站文章；
4、思路清晰，善于主动思考，有创新、能独立分析和解决问题，具有良好的沟通能力和团队合作精神；
5、具备漏洞分析、**木马分析、Web攻防、威胁情报挖掘、反APT攻击相关工作经验优先
6、有接触过SOC（安全运行中心）经验的优先；
7、熟悉IBM、Splunk Soar平台的优先；"""]]

# <a>https://www.cnblogs.com/mokundong/p/tfiwf.html<a>

    
# 1, TFIDF
## $ TF-IDF 是一个统计方法，用来评估某个词语对于一个文件集或文档库中的其中一份文件的重要程度。TF-IDF $
## $ 实际上是两个词组 Term Frequency 和 Inverse Document Frequency 的总称，两者缩写为 TF 和 IDF 分别代表了词频和逆向文档频率。$
    
## $ 词频 TF 计算了一个单词在文档中出现的次数，它认为一个单词的重要性和它在文档中出现的次数呈正比。$
## $ 逆向文档频率 IDF，是指一个单词在文档中的区分度。$
## $ 它认为一个单词出现在的文档数越少，就越能通过这个单词把该文档和其他文档区分开。$
## $ IDF 越大就代表该单词的区分度越大。所以 TF-IDF 实际上是词频 TF 和逆向文档频率 IDF 的乘积. $

## $ 这样我们倾向于找到 TF 和 IDF 取值都高的单词作为区分，$

## $ 即这个单词在一个文档中出现的次数多，同时又很少出现在其他文档中。这样的单词适合用于分类。$

## $ 但是TF-IDF在信息抽取上就不是很好的效果，因为tf-idf的idf在分母上采用了该单词的文档数，$
## $ 所以就造成了文档一些关键词在特征表现上过于平滑 $

# $$ 词频TF = 单词出现的次数/该文档的总单词数 $$
# $$ 逆向文档的频率IDF = log(文档总数/该单词出现的文档数+1) $$
# $$ 词频TF * 逆向文档的频率IDF $$

    
# 2, TF-IWF
## 此处的 $T F$ 与 $T F-I D F$ 中意义一样，表示词频：

# $$ t f_{i j}=\frac{n_{i, j}}{\sum_{k} n_{k, j}} $$

## 上式中分子 $n_{i, j}$ 表示词语 $t_{i}$ 在文本 $j$ 中的频数，分母 $\sum_{k} n_{k, j}$ 表示文档 $j$ 中所有词汇量总和，即是说：
    
# $$ T F_{w}=\frac{\text { 给定词 } w \text { 在当前文章出现的次数 }}{\text { 当前文章中的总词量 }} $$
    
## $ 不同之处在于 $I W F$ 部分，定义为：$

# $$ i w f_{i}=\log \frac{\sum_{i=1}^{m} n t_{i}}{n t_{i}} $$

## 上式中分子 $\sum_{i=1}^{m} n t_{i}$ 表示语料库中所有词语的频数之和，分母 $n t_{i}$ 表示词语 $t_{i}$ 在语料库中的总频数，即：
    
# $$ I W F_{i}=\frac{\text { 语料库中所有词语的频数 }}{\text { 给定词 } w \text { 在语料库中出现的频数和 }} $$
    
## 因此, $T F-I W F$ 定义为：

# $$ T F-I W F_{i, j} \rightarrow t f_{i, j} \times i w f_{i}=\frac{n_{i, j}}{\sum_{k} n_{k, j}} \times \log \frac{\sum_{i=1}^{m} n t_{i}}{n t_{i}} $$


In [ ]:
def TFIWF_function(original_list: (list, ndarray, set, tuple)
                   , filter_stop_words=True):
    '''
    Args:
        word_punct_tokens: 分词后的文章列表
        vocabulary: 词汇表

    Returns: tf 矩阵 横向量为词汇表

    

    '''
    word_punct_tokenizer = data_preprocessing_for_tfidf_function(original_list, filter_stop_words=filter_stop_words)
    word_punct_tokens, vocabulary = word_punct_tokenizer["word_punct_tokenizer"] , word_punct_tokenizer["vocabulary_set"]
    m, nti = len(word_punct_tokens), len(vocabulary)
    init_TF = zeros((m, nti))
    init_IWF = zeros(nti)
    init_counter_words_for_each_document = zeros(m)
    
    for No, paper_tokens in tqdm(word_punct_tokens.items(),"tfidf训练"):
        vocabulary_of_each_document = len(paper_tokens)
        init_TF += array([paper_tokens.count(word) for word in vocabulary])
        init_IWF += array([word in paper_tokens and 1 or 0 for word in vocabulary])
        init_counter_words_for_each_document[No] = vocabulary_of_each_document
        #print(init_IWF)

    TF = (init_TF.T / init_counter_words_for_each_document).T

    IWF = log(nti / (init_IWF))

    return {"TF-IWF": TF * IWF, "TF": TF, "IDF": IWF, "init_TF": init_TF[0],"document_count": m,"vocabulary_from_TF-IDF": vocabulary ,"counter_vector":init_IWF }
            #, "words_counter": init_IwF.dot(ones(nti)),
              # 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


TFIWF = TFIWF_function(case_text)
plt.figure(figsize=(26,9))
for vec in asnumpy(TFIWF["TF-IWF"]):
    plt.plot(vec)
#plt.xlabel([str(i) for i in TFIWF["vocabulary_from_TF-IDF"]])
#plt.ylabel(range(len(TFIWF["vocabulary_from_TF-IDF"])))
plt.title("TF-IDF",c='black')
plt.show()

# 对TF-IDF效果

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

plt.figure(figsize=(26,9))
for vec in asnumpy(TFIDF_function(case_text)["TF-IDF"]):
    plt.plot(vec)
plt.title("TF-IDF",c='black')
plt.show()